<a href="https://colab.research.google.com/github/Dimildizio/DS_course/blob/main/Neural_networks/Transfer_learning/imagenette_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Transfer learning

## Downloading and importing libs

In [1]:
from IPython import display

In [2]:
!pip install -U albumentations
!pip install -q --upgrade wandb
!pip install timm

display.clear_output()

In [3]:
import os
import wandb
import shutil
from pathlib import Path

import numpy as np
import random

import cv2
from tqdm import tqdm
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms
from torch.optim.lr_scheduler import StepLR

try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo. Installing...")
    !pip install -q torchinfo
    from torchinfo import summary

import timm

import albumentations as A
import albumentations.pytorch as AP

from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90, Resize, RandomCrop,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, RandomBrightnessContrast, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, Flip, OneOf, Compose, Rotate, RandomScale, RandomGridShuffle,
    RandomContrast, RandomGamma, RandomBrightness, CenterCrop, VerticalFlip, ColorJitter,
    ChannelShuffle, InvertImg, RGBShift, ElasticTransform, Equalize, RandomResizedCrop, ChannelDropout
)

import matplotlib.pyplot as plt
%matplotlib inline

# Setting things up

##Creating config class to utilize global variables

In [19]:
class CFG:

    num_workers=2
    model_name = 'inception_v3'
    size = 224
    scheduler = 'StepLR'
    epochs = 15
    step_size = 20 # StepLR
    gamma = 0.1 # StepLR
    lr = 3e-4
    min_lr = 1e-6  #
    batch_size = 8
    seed = 42

    api = ""
    project = "imagenette"
    entity = "dimildizio"
    wandb = True

    @classmethod
    def _set_model_name(cls, name):
      cls.model_name = name

    @classmethod
    def _set_lr(cls, lr):
      cls.lr = lr

    @classmethod
    def _set_epochs(cls, epoch):
      cls.epochs = epoch

    @classmethod
    def _switchflag(cls):
      cls.wandb = False if cls.wandb else True
      print(f'Wandb logging: {cls.wandb}')

    @classmethod
    @ property
    def _get_config(cls):
      return {key:value for key, value in CFG.__dict__.items() if (
                key[:1]!= '_' and key not in ('api', 'project', 'entity', 'wandb'))}


In [5]:
 CFG._get_config

{'num_workers': 2,
 'model_name': 'inception_v3',
 'size': 224,
 'scheduler': 'StepLR',
 'epochs': 15,
 'step_size': 20,
 'gamma': 0.1,
 'lr': 0.0003,
 'min_lr': 1e-06,
 'batch_size': 8,
 'seed': 42}

## Prepare data transformations - augmentation, normalization

### Modify torch.utils.data.Dataset class for augmented data

In [6]:
class MakeDataset(torch.utils.data.Dataset):
    def __init__(self, files, transform=None):
        super().__init__()
        self.files = files
        self.labels = [path.parent.name for path in self.files]
        self.len_ = len(self.files)
        self.transform = transform
        self.classes = ['n01440764', 'n02102040', 'n02979186', 'n03000684', 'n03028079', 'n03394916',
                        'n03417042', 'n03425413', 'n03445777', 'n03888257']

    def __len__(self):
        return self.len_

    def __getitem__(self, index):
        image = cv2.imread(f'{self.files[index]}')
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        label = self.labels[index]
        for i in range(len(self.classes)):
          if label == self.classes[i]:
            y = i

        return image, y

### Augmentations, transformations, normalization

In [7]:
def get_transforms(*, data):

    if data == 'train':
        transforma = A.Compose([Resize(256,256), RandomCrop(CFG.size,CFG.size), Rotate(limit = 90, p=0.5),
                                    GaussNoise(p=0.5), A.Sharpen(p=0.5), ChannelShuffle(p=0.5),
                                    HorizontalFlip(p=0.5), A.Normalize(), AP.ToTensorV2()])
    elif data == 'valid':
        transforma = A.Compose([Resize(256,256), CenterCrop(CFG.size,CFG.size), A.Normalize(), AP.ToTensorV2()])
    return transforma

In [8]:
def get_loaders(train_df, val_df):
  train_loader = torch.utils.data.DataLoader(train_df, batch_size=CFG.batch_size,shuffle=True)
  valid_loader = torch.utils.data.DataLoader(val_df, batch_size = CFG.batch_size)

  print('Train and Valid datasets are loaded:\n')
  print('{:<7s}{:>10s}{:>10s}'.format('Dataset', 'Batches', 'Pictures')), print('-' * 28)
  print('{:<7s}{:>10d}{:>10d}'.format('Train', len(train_loader), len(train_df)))
  print('{:<7s}{:>10d}{:>10d}'.format('Valid', len(valid_loader), len(val_df)))
  return train_loader, valid_loader

# Downloading dataset and preparing data

In [9]:
dataset_path = 'imagenette2-160/'
if not os.path.exists(dataset_path):
    !wget https://s3.amazonaws.com/fast-ai-imageclas/imagenette2-160.tgz
    !tar zxvf imagenette2-160.tgz

TRAIN_DIR = Path('/content/imagenette2-160/train')
VAL_DIR = Path('/content/imagenette2-160/val')

train_files = sorted(list(TRAIN_DIR.rglob('*.JPEG')))
val_files = sorted(list(VAL_DIR.rglob('*.JPEG')))
display.clear_output()

In [10]:
train_dataset = MakeDataset(train_files, transform = get_transforms(data='train'))
val_dataset  = MakeDataset(val_files, transform = get_transforms(data='valid'))

In [11]:
train_loader, valid_loader = get_loaders(train_dataset, val_dataset)

Train and Valid datasets are loaded:

Dataset   Batches  Pictures
----------------------------
Train        1184      9469
Valid         491      3925


In [23]:
class Performer:
  trainloader = train_loader
  validloader = valid_loader



  @classmethod
  def get_device(cls):
    cuda = 'cuda' if torch.cuda.is_available() else 'cpu'
    device = torch.device(cuda)
    print(device)
    return device

  @classmethod
  def trainval_hist(cls):
    return {'train':[], 'val':[]}, {'train':[], 'val':[]}

  @classmethod
  def run_wandb(cls):
    if CFG.wandb:
      os.environ['WANDB_API_KEY'] = CFG.api
      wandb.init(project = CFG.project, entity = CFG.entity,
                 name = CFG.model_name, reinit = True, config = CFG._get_config)

  @classmethod
  def log_wandb(cls, name, loss, acc):
    if CFG.wandb:
      wandb.log({name+'_loss': loss, name+'_accuracy': acc})

  @classmethod
  def seed_everything(cls):
    seed = CFG.seed
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] =str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

  @classmethod
  def switch_trainval(cls, model, phase):
    if phase == 'train':
      model.train()
      return cls.trainloader
    else:
      model.eval()
      return cls.validloader

  @classmethod
  def train_val(cls, model, optimizer, loss = nn.CrossEntropyLoss(), scheduler = False):
    #pre-setup
    cls.seed_everything()
    cls.run_wandb()
    device = cls.get_device()
    #set local variables for all epochs
    loss_hist, acc_hist = cls.trainval_hist()
    best_acc = 0.
    best_weights = model.state_dict()

    for epoch in range(CFG.epochs):
      print(f'Epoch: {epoch+1}/{CFG.epochs}')
      #each epoch a phase of train and a phase of val
      for phase in ['train', 'val']:
        dataloader = cls.switch_trainval(model, phase)

        model.to(device)
        running_loss, running_acc = 0., 0.

        #set X, y
        for inputs, labels in tqdm(dataloader):
          inputs = inputs.to(device)
          labels = labels.to(device)
          #predict and adjust if train
          optimizer.zero_grad()
          with torch.set_grad_enabled(phase == 'train'):
            y_pred = model(inputs)
            loss_v = loss(y_pred, labels)
            if phase == 'train':
              loss_v.backward()
              optimizer.step()
          #calc train val stats
          running_loss += loss_v.item()
          running_acc += (y_pred.argmax(dim=1) == labels).float().mean().data.cpu().numpy()
        if phase == 'train' and scheduler:
          scheduler.step()
        #calc epoch stats
        epoch_loss = running_loss / len(dataloader)
        epoch_acc = running_acc / len(dataloader)
        cls.log_wandb(phase, epoch_loss, epoch_acc)
        display.clear_output()
        print(f'Epoch:{epoch+1}, loss:{round(epoch_loss,3)}, accuracy: {round(epoch_acc,3)}')
        #select best weights
        if phase == 'val' and epoch_acc > best_acc:
          best_acc = epoch_acc
          best_weights = model.state_dict()
        #write for history
        loss_hist[phase].append(epoch_loss)
        acc_hist[phase].append(epoch_acc)
    torch.save(best_weights, '/content/'+f'{CFG.model_name}_best.pth')
    return model, loss_hist, acc_hist


In [13]:
class SimpleModel(nn.Module):
    def __init__(self, num_classes):
        super(SimpleModel, self).__init__()

        # Define your layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(16 * 224 * 224, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x


In [14]:
CFG._set_model_name('SimpleModel')
CFG._set_lr(3e-3)

CFG._set_epochs(10)
print(CFG.epochs, CFG.lr)

10 0.003


In [21]:
CFG._switchflag()

Wandb logging: True


In [22]:
qq = SimpleModel(10)
optimizer = torch.optim.AdamW(qq.parameters(), CFG.lr)
Performer.train_val(qq, optimizer, scheduler = StepLR(optimizer, step_size=CFG.step_size, gamma=CFG.gamma))

Epoch:15, loss:1.922, accuracy: 0.354


(SimpleModel(
   (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (relu): ReLU()
   (flatten): Flatten(start_dim=1, end_dim=-1)
   (fc): Linear(in_features=802816, out_features=10, bias=True)
 ),
 {'train': [5.790056021582033,
   2.4837271909455994,
   2.2203714607937917,
   2.169712279948431,
   2.085303667227964,
   2.0654405330282612,
   2.022474389825318,
   2.0322642294136255,
   2.003359331271133,
   2.003437509105818,
   2.0088080401356154,
   1.9963812148953612,
   1.9838164703467407,
   1.9827070006144207,
   1.9843518790361043],
  'val': [1.9522622985776719,
   1.9033764765850152,
   1.8692474036255582,
   1.9008410462536782,
   1.9540871470377548,
   1.9426148291032328,
   1.939178070929774,
   1.9284272401493097,
   1.9078649209618812,
   1.9282657932603917,
   1.8962306041581567,
   1.9212714686170371,
   1.9480690486319439,
   1.8994587652552153,
   1.9223298510563107]},
 {'train': [0.23788006757260174,
   0.297043918921436,
   0.31399915541043

### Importing a model and freezing layers

In [27]:
namelist = ['resnet50', 'resnext101_32x8d', 'inception_v3', 'densenet169', 'efficientnet_b0']

In [62]:
class TransferModel(torch.nn.Module):
  def __init__(self, name='resnet50', value=10):
    super().__init__()
    self.model = timm.create_model(name, pretrained=True)
    self.freeze_me()
    self.give_head(name, value)

  def give_head(self, name, value):
    #sic!
    if 'effi' in name:
      self.classifier_head_eff(value)
    else:
      self.classifier_head(value)

  def classifier_head_eff(self, value):
    inputs = self.model.classifier.in_features
    #self.model.classifier = nn.Identity()
    self.model.classifier = nn.Linear(inputs, value)

  def classifier_head(self, value):
    inputs = self.model.fc.in_features
    self.model.fc = nn.Linear(inputs, value)

  def freeze_me(self):
    model_len = len(list(self.model.parameters()))
    for idx, param in enumerate(self.model.parameters()):
      param.requires_grad = False if idx < model_len-2 else True

  @property
  def params(self):
    layers = list(self.model.parameters())[:-2] + list(self.parameters())[-2:]
    return layers

  def forward(self, x):
    return self.model(x)

### Freezing all layers except the last one

In [54]:
model = TransferModel()
summary(model=model,
        input_size= (CFG.batch_size, 3, CFG.size, CFG.size),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20)

Layer (type:depth-idx)                        Input Shape          Output Shape         Param #              Trainable
TransferModel                                 [8, 3, 224, 224]     [8, 10]              --                   Partial
├─ResNet: 1-1                                 [8, 3, 224, 224]     [8, 10]              --                   Partial
│    └─Conv2d: 2-1                            [8, 3, 224, 224]     [8, 64, 112, 112]    (9,408)              False
│    └─BatchNorm2d: 2-2                       [8, 64, 112, 112]    [8, 64, 112, 112]    (128)                False
│    └─ReLU: 2-3                              [8, 64, 112, 112]    [8, 64, 112, 112]    --                   --
│    └─MaxPool2d: 2-4                         [8, 64, 112, 112]    [8, 64, 56, 56]      --                   --
│    └─Sequential: 2-5                        [8, 64, 56, 56]      [8, 256, 56, 56]     --                   False
│    │    └─Bottleneck: 3-1                   [8, 64, 56, 56]      [8, 256, 56

In [55]:
CFG._set_epochs(10)
CFG._set_lr(4e-4)
CFG._set_model_name(namelist[0])
CFG._get_config

{'num_workers': 2,
 'model_name': 'resnet50',
 'size': 224,
 'scheduler': 'StepLR',
 'epochs': 10,
 'step_size': 20,
 'gamma': 0.1,
 'lr': 0.0004,
 'min_lr': 1e-06,
 'batch_size': 8,
 'seed': 42}

In [58]:
CFG._switchflag()

Wandb logging: True


In [59]:
optimizer = torch.optim.AdamW(model.params, lr = CFG.lr)
scheduler = StepLR(optimizer, step_size=CFG.step_size, gamma=CFG.gamma)
Performer.train_val(model, optimizer, scheduler = scheduler)

Epoch:10, loss:0.101, accuracy: 0.976


(TransferModel(
   (model): ResNet(
     (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
     (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (act1): ReLU(inplace=True)
     (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
     (layer1): Sequential(
       (0): Bottleneck(
         (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
         (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
         (act1): ReLU(inplace=True)
         (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
         (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
         (drop_block): Identity()
         (act2): ReLU(inplace=True)
         (aa): Identity()
         (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
         (bn3): BatchNorm2

In [ ]:
CFG._switchflag()